In [20]:
import pandas as pd
from data_tweak.converters import convert_race, convert_sex

merged_chunks = []

officers_prof_df = pd.read_csv("../files/profiles/officer_roster.csv")
convert_sex(officers_prof_df, ["off_sex"])
convert_race(officers_prof_df, ["off_race"])

with pd.read_csv("../files/events/assignment_cleaned.csv", chunksize=10000, date_parser=pd.to_datetime, parse_dates=["shift_start", "shift_end_corrected"]) as reader:
  for chunk in reader:
    chunk.insert(0, "assignment_id", chunk.index)
    chunk.reset_index(drop=True, inplace=True)
    convert_sex(chunk, ["off_sex"])
    convert_race(chunk, ["off_race"])
    
    merged_chunk = pd.merge(chunk, officers_prof_df, on=["off_last_name","off_first_name","off_middle_initial","off_sex","off_year_of_birth","off_appointed"], how="left")
    merged_chunk.sort_values(by=["shift_start", "assignment_id", "off_last_name", "off_first_name", "off_middle_initial"], inplace=True)
    merged_chunks.append(merged_chunk)

merged_df = pd.concat(merged_chunks)
grouped = merged_df.groupby("assignment_id").size().sort_values(ascending=False).to_frame("size").reset_index()
grouped[grouped["size"] > 1]


KeyboardInterrupt: 

In [ ]:
merged_df

,assignment_id,Unnamed: 0,unit,watch,beat,car_number,shift_start,start_time,shift_end,end_time,...,off_uniq_id,off_race_y,ranks_held,off_star_0,off_star_1,off_star_2,off_star_3,off_star_4,off_star_5,off_source
3258,3258,3259,610,1,5159,4143,2015-01-01,0.0,2015-01-01 09:00:00,900.0,...,17421,WHITE,POLICE OFFICER ASSIGNED AS DETECTIVE,21001.0,NaN,NaN,NaN,NaN,NaN,assignments_file
3259,3259,3260,610,1,5116,2140,2015-01-01,0.0,2015-01-01 08:00:00,800.0,...,17877,BLACK,POLICE OFFICER ASSIGNED AS DETECTIVE,20260.0,21848.0,NaN,NaN,NaN,NaN,assignments_file
3261,3261,3262,610,1,5103,NaN,2015-01-01,0.0,2015-01-01 09:00:00,900.0,...,20252,WHITE,POLICE OFFICER ASSIGNED AS DETECTIVE,20432.0,NaN,NaN,NaN,NaN,NaN,assignments_file
3262,3262,3263,610,1,5103,NaN,2015-01-01,0.0,2015-01-01 09:00:00,900.0,...,21504,BLACK,POLICE OFFICER ASSIGNED AS DETECTIVE,20397.0,NaN,NaN,NaN,NaN,NaN,assignments_file
3263,3263,3264,610,1,5135,3824,2015-01-01,0.0,2015-01-01 09:00:00,900.0,...,22313,BLACK,POLICE OFFICER ASSIGNED AS DETECTIVE,21442.0,NaN,NaN,NaN,NaN,NaN,assignments_file
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9420,199415,199416,277,2,NaN,NaN,NaT,NaN,NaN,NaN,...,29645,WHITE,PO ASGN EVID. TECHNI,7839.0,NaN,NaN,NaN,NaN,NaN,assignments_file
9421,199416,199417,277,2,NaN,NaN,NaT,NaN,NaN,NaN,...,31753,WHITE,PO ASGN EVID. TECHNI,3122.0,NaN,NaN,NaN,NaN,NaN,assignments_file
9425,199420,199421,50,4,7262A,PD21,NaT,NaN,NaN,NaN,...,4039,WHITE,SERGEANT OF POLICE,1905.0,12135.0,NaN,NaN,NaN,NaN,assignments_file
9776,199771,199772,189,2,6245F,1619,NaT,NaN,NaN,NaN,...,12103,WHITE,SERGEANT OF POLICE,1312.0,8775.0,NaN,NaN,NaN,NaN,assignments_file
